In [ ]:
def gaspari_cohn(array,distance,center):
    """
    NAME 
        bfn_gaspari_cohn

    DESCRIPTION 
        Gaspari-Cohn function. @vbellemin.
        
        Args: 
            array : array of value whose the Gaspari-Cohn function will be applied
            center : centered value of the function 
            distance : Distance above which the return values are zeros


        Returns:  smoothed values 
            
    """ 
    if type(array) is float or type(array) is int:
        array = np.array([array])
    else:
        array = array
    if distance<=0:
        return np.zeros_like(array)
    else:
        array = 2*np.abs(array-center*np.ones_like(array))/distance
        gp = np.zeros_like(array)
        i= np.where(array<=1.)[0]
        gp[i]=-0.25*array[i]**5+0.5*array[i]**4+0.625*array[i]**3-5./3.*array[i]**2+1.
        i =np.where((array>1.)*(array<=2.))[0]
        gp[i] = 1./12.*array[i]**5-0.5*array[i]**4+0.625*array[i]**3+5./3.*array[i]**2-5.*array[i]+4.-2./3./array[i]
        #if type(r) is float:
        #    gp = gp[0]
    return gp

In [ ]:
def extract_it(array_ssh,wint,H): 
    array_ssh=array_ssh.values
    ssh_extended = np.concatenate((np.flip(array_ssh),
                                   array_ssh,
                                   np.flip(array_ssh)))
    ssh_win = wint * ssh_extended 
    ssh_f_t = fp.fft(ssh_win)
    ssh_f_filtered =  H * ssh_f_t
    ssh_filtered = np.real(fp.ifft(ssh_f_filtered))[nt:2*nt]
    del array_ssh
    return ssh_filtered

In [ ]:
def create_cartesian_grid(latitude, longitude, dx, extra_pixels=7):
    """ 
    Creates a cartesian grid (regular in distance, kilometers) from a geodesic latitude, longitude grid. 
    The new grid is expressed in latitude, longitude coordinates.

    Parameters
    ----------
    longitude : numpy ndarray 
        Vector of longitude for geodesic input grid. 
    latitude : numpy ndarray 
        Vector of latitude for geodesic input grid. 
    dx : float 
        Grid spacing in kilometers. 
    extra_pixels : int, optional
        Number of extra pixels to add on each side of the grid. Default is 2.

    Returns
    -------
    ENSLAT2D : 
        2-D numpy ndarray of the latitudes of the points of the cartesian grid 
    ENSLON2D : 
        2-D numpy ndarray of the longitudes of the points of the cartesian grid 
    """
    km2deg = 1 / 111

    # Extend the latitude range by extra_pixels grid points on each side
    ENSLAT = np.arange(latitude[0] - extra_pixels * dx * km2deg, latitude[-1] + (extra_pixels + 1) * dx * km2deg, dx * km2deg)
    range_lon = longitude[-1] - longitude[0]
    
    # Extend by the number of extra pixels on each side
    if longitude.size % 2 == 0:
        nstep_lon = floor(range_lon / (dx * km2deg)) + 2 * (extra_pixels + 1)  
    else:
        nstep_lon = ceil(range_lon / (dx * km2deg)) + 2 * (extra_pixels + 1) 

    ENSLAT2D = np.repeat(np.expand_dims(ENSLAT, axis=1), axis=1, repeats=nstep_lon)

    # ENSEMBLE OF LONGITUDES
    mid_lon = (longitude[-1] + longitude[0]) / 2
    ENSLON2D = np.zeros_like(ENSLAT2D)

    for i in range(len(ENSLAT)):
        d_lon = dx * km2deg * (np.cos(np.pi * latitude[0] / 180) / np.cos(np.pi * ENSLAT[i] / 180))
        d_lon_range = np.array([j * d_lon for j in range(1, int(nstep_lon / 2) + 1)])
        lon_left = np.flip(mid_lon - d_lon_range)
        lon_right = mid_lon + d_lon_range
        ENSLON2D[i, :] = np.concatenate((lon_left, [mid_lon], lon_right))[:nstep_lon]

    return ENSLAT2D, ENSLON2D, ENSLAT2D.shape[0], ENSLAT2D.shape[1]


In [ ]:
# Function to process a single time step
def process_time_step(time_idx, ssh_it, dx):
    # Select the time step
    first_date_ssh = ssh_it.isel(time_counter=time_idx)
    
    # Create a cartesian grid
    ENSLAT2D, ENSLON2D, i_lat, i_lon = create_cartesian_grid(first_date_ssh.nav_lat[:, 0].values, first_date_ssh.nav_lon[0, :].values, dx)
    
    # Flatten the grids
    ENSLAT2D_flat = ENSLAT2D.flatten()
    ENSLON2D_flat = ENSLON2D.flatten()
    
    # Prepare the points and values for interpolation
    points = np.column_stack((first_date_ssh.nav_lat.data.flatten(), first_date_ssh.nav_lon.data.flatten()))
    values = first_date_ssh.ssh_it.data.flatten()
    target_grid = np.column_stack((ENSLAT2D_flat, ENSLON2D_flat))
    
    # Interpolate the data to the new grid
    array_cart_ssh = griddata(points, values, target_grid, method='linear')
    
    # Reshape the interpolated data to the grid shape
    array_cart_ssh = array_cart_ssh.reshape(ENSLAT2D.shape)
    
    # Filling extra pixels with the Gauss-Seidel method
    x_axis = Axis(np.arange(i_lon))
    y_axis = Axis(np.arange(i_lat))
    grid = Grid2D(y_axis, x_axis, array_cart_ssh.reshape((i_lat, i_lon)))
    has_converged, filled = fill.gauss_seidel(grid)
    
    # Create a new DataArray with the filled data
    cart_it = xr.DataArray(
        data=filled,
        dims=["y", "x"],
        coords=dict(
            y=(["y"], np.arange(i_lat)),
            x=(["x"], np.arange(i_lon)),
            time_counter=first_date_ssh.time_counter
        )
    )
    
    return cart_it

In [ ]:
def plot_spectrum(res,k):
    
    fig, ax = plt.subplots(1,2)

    k1 = k[0]#0.0070
    k2 = k[1]#0.0126
    k3 = k[2]#0.0191
    k4 = k[3]#0.0269

    ax[0].plot(res.freq_r.values,res.values)
    ax[0].set_xlim(0.03,0)
    ax[0].set_xlabel("Wavenumber [km-1]")
    ax[0].axvline(k1,c='red',linestyle=':')
    ax[0].axvline(k2,c='red',linestyle=':')
    ax[0].axvline(k3,c='red',linestyle=':')
    ax[0].axvline(k4,c='red',linestyle=':')
    ax[0].axvline(k1/2,c='red',linestyle='-')
    ax[0].axvline((k1+k2)/2,c='red',linestyle='-')
    ax[0].axvline((k2+k3)/2,c='red',linestyle='-')
    ax[0].axvline((k3+k4)/2,c='red',linestyle='-')
    
    ax[1].plot(1/res.freq_r.values,res.values)
    ax[1].set_xlim(0,200)
    ax[1].set_xlabel("Wavelength [km]")

    fig.suptitle("Isotropic Power Spectrum of Internal Tides")